# Morphological analysis

- create list of 50 signs and their synonyms from the course
- find a way to group two words into one sign
- read about Morphological analysis to make the pairs to solve the problem of verbs & nouns
- find way to apply rules like removing  على
- find difference between كتب which is book and wrote 




# very important

- كلمة بيتك وبيتى او اى كلمة فيها ملكية ف وقتها حولها للكلمة الاصلية  اللى هيا هنا هتكون بيت وبعدين اعمل اشارة ملكية او اشارة للمخاطب اللى هيا بتكون انك بتشاور على نفسك او بتشاور امامك وايدك مقفولة
- المذكر والمؤنث فى كلمات عديدة يتم باستعمال اشارة الكلمة نفسها تليها اشارة لتحديد المذكر او المؤنث
- some words don't follow the previous rule so i have to seperate between the words that follow these rules and the words that don't
- اشارة اخت عبارة عن اشارة اخ + اشارة امرأة ونفس الامر لكمات عديدة
- فكرة الملكية بنتطبق برضو على الجمع انة اشارة الكلمة وبعدين جمع او لا 
- مش بحب بتتحول ل بحب لا  ودة بينطبق على كلمات كثير ف حاول تشوف حل لانك تعكس الترتيب دة 
- بعض الاوقات كلمة الجمع مش بتتحط عشان مش مهمة اوى ف حتى لو الكلمة جمع انت بتعمل اشارة المفرد من غير ما تحط حاجة تبين ان قصدك الجمع ودة بيتفهم من السياق
- بعض الاوقات الازمنة مهمة واوقات تانى لا ف مثلا شرب و بشرب وهشرب ممكن يترجموا لنفس الاشارة فى الاخر او ممكن يترجموا ل شرب ماضى او شرب الان او شرب مستقبل واللى بيكون اشارة بتعبر عن الحاجة ووراها اشارة بتعبر عن الزمن
- فى لغة الاشارة بتعرف الفاعل من الفعل ان الفاعل بيكون عبارة عن اشارة شخص + اشارة الفعل نفسة

# confusing words so far
-  غيرة it could me jealousy or some on else
- قلب heart or middle ex : فى قلب الاحداث او قلب انسان
- علم flag or science
- كتب wrote or books
- مركز focus or position

# try to translate these examples
- السلام عليكم ورحمة الله وبركاتة
- انت اسمك اية 
- ما هو اسمك
- كيف حالك
- انت عامل اية 
- ازيك
- ما شاء الله
- الحمد لله
- انت اصم 
- انا شوفتك 
- انت بتعلمنى كثير ربنا يحفظك
- ما هى وظيفتك
- انت بتشتغل اية 
- عندك كام سنة 
- انت متزوج ؟ 
- انا بعمل كل حاجة لنفسى
- مش بحب اساعد حد
- انت شجاع لكن متكبر
- انت ازاى ضعيف ومكتبر
- قل الصدق دائما ولا تكذب
- انا كنت مشوش بسبب الامتحان
- انا بكرة المدرسة بس لازم اروح
- انا بكرهك
- انا بحبك
- انا رحت المكتبة امبارح وقرات كتاب جميل -- الجملة دة ممكن تعمل مشكلة بسبب ان كلمة جميل ليها اشارة محتلفة سواء المقصود بيها جميل لشخص او لكتاب عشان لو لشخص هيبقى اشارة وش جميل ولو ل لجماد ف اشارة تانية ف منطقيا متنفعش
- الساعة كام ؟
- لماذا نحن هنا
- الحكومة تساعد الشعب 
- حاول مسئول حكومى التخلى عن منصبة فى الاحلام
- انا طالب فى المرحلة فى الابتدائة
- انت طالب اعدادى
- انا بكرة الطلاب
- انا بكرة المعلمون
- لماذا نحن هنا تانى
- احمد اخد البراية بتاعتى من غير ما يقول عشان هو حيوان وخنزير
- انا سقطت فى امتحان العربى بس نجحت فى امتحان العلوم 
- انا نجحت فى الثانوية وهدخل الجامعة
- انا بكرة كلية هندسة
- انا بكرة الجامعة
- تبا لهندسة
- احنا لية هنا ؟
- انا بحب اتعلم بس مش بحب المدرسة
- الخيل والليل والبيداء تعرفنى والسيف والرمح والقرطاس والقلم --- لا انا بهزر وهمسح دة قريب اصلا 
- انا زعلان وانت سعيد تبا لك انا غيران 
- انا فرحان عشان الجامعة اتفجرت
- ما هى طريقة عمل البيتزا
- ازاى تعمل بيتزا 
- ازااااااى
- انت لية مش بترد
- انت جبان ؟ 
- انا بحب الرياضيات 
- انا نجحت لما جبت 70% الحمد لله
- يا فاشل ال 50% دة مش هتدخلك حتى البيت
- قال الخنزير : الامتحان سهل جدا 
- 

# convert symbols to actual arabic

In [24]:
import re
import os
import json
from collections import defaultdict

In [59]:
# i only used this to reorganize the numbers (ids) of the json file
num,name = "010_محافظات".split("_")

sign_language_data = os.path.join("signLanguageData",f"{num}_{name}.json")

counter=1
with open(sign_language_data, 'r') as json_file:
    json_data = json.load(json_file)
    for key in json_data:
        print(f' "sign_{num}_{counter}":{json_data[key]}',",",sep="")
        counter+=1
    

 "sign_010_1":{'word': ['محافظة'], 'description': '', 'video_num': 16, 'sign_time': '12:15'},
 "sign_010_2":{'word': ['مكان'], 'description': '', 'video_num': 16, 'sign_time': '7:22'},
 "sign_010_3":{'word': ['عين شمس'], 'description': '', 'video_num': 16, 'sign_time': '1:27'},
 "sign_010_4":{'word': ['المرج'], 'description': '', 'video_num': 16, 'sign_time': '1:54'},
 "sign_010_5":{'word': ['المرج الجديدة'], 'description': '', 'video_num': 16, 'sign_time': '2:08'},
 "sign_010_6":{'word': ['القاهرة'], 'description': '', 'video_num': 16, 'sign_time': '2:18'},
 "sign_010_7":{'word': ['حلمية الزيتون'], 'description': '', 'video_num': 16, 'sign_time': '2:23'},
 "sign_010_8":{'word': ['المطرية'], 'description': '', 'video_num': 16, 'sign_time': '2:35'},
 "sign_010_9":{'word': ['المعادي'], 'description': '', 'video_num': 16, 'sign_time': '2:53'},
 "sign_010_10":{'word': ['المهندسين'], 'description': '', 'video_num': 16, 'sign_time': '3:14'},
 "sign_010_11":{'word': ['الهرم'], 'description': 

# text processing

In [19]:
import re
import os
from collections import defaultdict


# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","_","-","/",".","،"," و ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!','  ']
    replace = ["ا","ا","ا"," "," ","","",""," و","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ',' ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for a,b in zip(search,replace):
        text = text.replace(a,b)
    
    #trim    
    text = text.strip()
    return [word.strip() for word in text.split(" ")]

    return text

# lemmatization

In [2]:
#https://farasa.qcri.org/lemmatization/

import json
import requests

URL = 'https://farasa.qcri.org/webapi/lemmatization/'
API_KEY = "mgmfGsWLmIFJznbMaC"
LEMMATIZATION_FILE = "lemmatization.json"

with open(LEMMATIZATION_FILE, 'r') as json_file:
    lemmatization_table = json.load(json_file)


def fetch(arr):    
    text = " ".join(arr)
    payload = {'text': text, 'api_key': API_KEY}
    data = requests.post(URL, data=payload)
    result = json.loads(data.text)
    return result['text']

def lemmatization(arr):
    words = []
    for word in arr:
        if(word not in lemmatization_table):
            words.append(word)
    if(len(words)>0):
        lemmatizated_words = fetch(words)
        for original,lemmaitaized in zip(words,lemmatizated_words):
            lemmatization_table[original] = lemmaitaized
    
    
        with open(LEMMATIZATION_FILE, 'w') as write:
            json.dump(lemmatization_table,write)
    
    final_results = []
    for word in arr:
        final_results.append(lemmatization_table[word])
    
    return final_results

# synonyms

In [37]:
import os
import json

sign_language_words = os.path.join("signLanguageWords","تعارف.txt")
sign_language_data = os.path.join("signLanguageData","تعارف.json")

json_data = {}
existing_words = {}
counter = 0;

with open(sign_language_data, 'r') as json_file:
    json_data = json.load(json_file)
    counter = len(json_data)
    for key,value in json_data.items():
        existing_words[value['word']] = 1;


    
with open(sign_language_words,"r") as f:
    lis = f.read().split("\n")
    
    for word in lis:
        if(word not in existing_words):
            json_data[f'sign_{counter}'] = {"word":word}
            counter+=1
    with open(sign_language_data, 'w') as json_file:
        json.dump(json_data,json_file)




for key,value in json_data.items():
    print(key,value['word'])

sign_0 تعارف
sign_1 السلام
sign_2 عليكم
sign_3 رحمه
sign_4 الله
sign_5 بركاته


In [16]:
def translate(arr):
    final = []
    for word in arr:
        if word in synonyms_table:
            final.append(synonyms_table[word])
        else:
            for letter in word:
                final.append(letter)
    return final

In [18]:
# ex ; we don't have سار but we  have ذهب 
# we don't have طريق so it spelled it 
text = clean_str("السلام عليكم ورحمه الله وبركاته")
results = lemmatization(text)
translate(results)

['س',
 'ل',
 'ا',
 'م',
 'ع',
 'ل',
 'ى',
 'ر',
 'ح',
 'م',
 'ا',
 'ل',
 'ل',
 'ه',
 'ب',
 'ر',
 'ك',
 'ة']

# steaming (might be used in the future)

In [20]:
import nltk

In [21]:
# https://www.nltk.org/api/nltk.stem.isri.html
obj = nltk.stem.isri.ISRIStemmer()


In [29]:
for i in ["الخير"]:
    print(obj.stem(i))

خير


In [17]:
for i in ["الاستنشاق","المحبة","الرحمة","يذهب","السما"]:
    print(obj.stem(i))

نشق
حبة
رحم
ذهب
سما
